In [ ]:
library(ggplot2)
library(reshape2)
library(gridExtra)
library(factoextra)
library(FactoMineR)
library(plotly)
library(ggmap)

## Data

In [ ]:
load <- read.csv("data/velibLoading.csv", sep=" ") # Reading data of the Velib loading
head(load)


In [ ]:
coord = read.csv('data/velibCoord.csv', sep = " ") # Reading coordinates for each loading
head(coord)

### Data cleaning

In [ ]:
cat('Shape of "load" :', dim(load))
number_of_missing_values_load <- sum(is.na(load))
number_of_duplicate_values_load <- sum(duplicated(load))
cat('\nNumber of missing values "load"', number_of_missing_values_load)
cat('\nNumber of duplicate values "load":', number_of_duplicate_values_load)

cat('\nShape of "coord" :', dim(coord))
number_of_missing_values_coord <- sum(is.na(coord))
number_of_duplicate_values_coord <- sum(duplicated(coord))
cat('\nNumber of missing values "coord"', number_of_missing_values_coord)
cat('\nNumber of duplicate values "coord":', number_of_duplicate_values_coord)


## Descriptive part

In [ ]:
station_names <- table(coord$names)
station_names <- sort(station_names,decreasing=TRUE)
head(station_names)
indexes <- which(station_names>1)
print(station_names[indexes])
cat("\nNumber of stations with multiple occurences in coord:", length(station_names[indexes]))

In [ ]:
name_checkup <- c(' PORTE DES LILAS', ' CLICHY')
multiple_stat <- coord$names %in% name_checkup
stat_rel <- coord[multiple_stat,]
print(stat_rel)

In [ ]:
fig <- qmplot(data=stat_rel, longitude, latitude, color = names, zoom = 16) + 
    labs(title = paste('Placement of stations with multiple occurances:',paste(name_checkup, collapse = ',')))
fig

Looking at two different stations with multiple occurences, we can see that they are really close geographically. We can suppose that the velib there are often used. Let's take a look at the loading of the station Porte des lilas .


In [ ]:
time_tick = 1 + 24*(0:6) 

ind <- which(coord$names == " PORTE DES LILAS")
i1 <- ind[1]

df = melt(load[i1,])  
df$time_range = 1:ncol(load)

p1 = ggplot(df, aes(x=time_range, y=value)) + geom_line(col="darkorchid") +
    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
    labs(title=coord$names[i1])

i2 <- ind[2]
df = melt(load[i2,])
df$time_range = 1:ncol(load)

p2 = ggplot(df, aes(x=time_range, y=value)) + geom_line(col="darkorchid") +
    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
    labs(title=coord$names[i2])

i3 <- ind[3]

df = melt(load[i3,]) 
df$time_range = 1:ncol(load)

p3 = ggplot(df, aes(x=time_range, y=value)) + geom_line(col="darkorchid") +
    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
    labs(title=coord$names[i3])

p <- list(p1,p2,p3)
do.call(grid.arrange,p)

Indeed, we can clearly see that the loading of those station are often near 0. It moves a lot so we can imagine that the velib are put there the morning and directly used by the inhabitants to go to work. We can explain that there is only one time in the day where the loading is high by the fact that this station is on a hill. We can suppose that the people are using the velib to go down to work and then take the public transport to go back up. Let's take a look at the loading of the station Clichy which is not on a hill.


In [ ]:
ind <- which(coord$names == " CLICHY")
i1 <- ind[1]

df = melt(load[i1,])  
df$time_range = 1:ncol(load)

p1 = ggplot(df, aes(x=time_range, y=value)) + geom_line(col="darkorchid") +
    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
    labs(title=coord$names[i1])

i2 <- ind[2]
df = melt(load[i2,])
df$time_range = 1:ncol(load)

p2 = ggplot(df, aes(x=time_range, y=value)) + geom_line(col="darkorchid") +
    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
    labs(title=coord$names[i2])


p <- list(p1,p2)
do.call(grid.arrange,p)

It is really more difficult to interpret the loading of this station.


In [ ]:
print('--- Average loading ---')
mean <- rowMeans(load) # Average per station
tot_mean <- mean(mean)
print(round(tot_mean,4))

# --- #
print(" ")
print('--- Least loaded station, on average ---')
print(paste(coord[which.min(mean),],round (min(mean),4)))
# --- 

print('')

print('--- Fullest loaded station, on average ---')
print(paste(coord[which.max(mean),],round (max(mean),4)))

# --- 
print("--- Average loading of station CLICHY ---")
print((mean[i1]+mean[i2])/2)


The average loading of the stations Clichy is clearly lower than the average loading of all the stations. The velib there are then often used. It explains the fact that there is two stations at the same place there. However, we can see that the station Hornet (Bagnolet) is often empty but is not part of the station with multiple occurrences. It would maybe be useful to add one more. On the other hand, some stations are not that useful. Indeed, the station Insurrection aout 1944 (Ivry) is almost full at every hour of the day, with an average loading of almost 92%. Let's take a closer look.


In [ ]:
ind <- which(coord$names == " INSURRECTION AOUT 1944 (IVRY)")
i <- ind[1]

df = melt(load[i,])  
df$time_range = 1:ncol(load)

ggplot(df, aes(x=time_range, y=value)) + geom_line(col="darkorchid") +
    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
    labs(title=coord$names[i])

min(load[i,])
```

We can now see that the lower loading is almost of 57% on wednesday but it is hard to explain why. Except that day, it does not go under 70%. Let's now take a look at the entire data frame.


In [ ]:
time_ticks <- seq(0,168, by = 24)

options(repr.plot.width = 15, repr.plot.height = 6)
bp <- boxplot(load, xlab = "Time", ylab = "Loading", col = "blue", medcol ="violet")
abline(v = time_ticks, col = "orange", lwd = 4, lty = "dotted")

We can see that the behavior of stations loading is similar every weekday.The median load rate remains between 20% and 40%, so we can deduce that there is an imbalance in the use of the various stations.


In [ ]:
mean_per_hour_per_day = colMeans(load)
mean_per_hour_per_day = matrix(mean_per_hour_per_day, nrow = 24)
mean_per_hour         = rowMeans(mean_per_hour_per_day)

# --- #

mean_per_hour_per_day            = as.data.frame(mean_per_hour_per_day)
colnames(mean_per_hour_per_day)  = list("Monday", "Tuesday", "Wednesday","Thursday", "Friday", "Saturday", "Sunday")
mean_per_hour_per_day$time_range = c(1:24)
mean_per_hour_per_day            = melt(mean_per_hour_per_day, id='time_range', variable.name='Days')

mean_per_hour            = as.data.frame(mean_per_hour)
colnames(mean_per_hour)  = list("Weekly")
mean_per_hour$time_range = c(1:24)

# --- #

options(repr.plot.width = 15, repr.plot.height = 10)

ggplot() + 
    geom_line(data=mean_per_hour_per_day, aes(x=time_range, y=value, color=Days)) + 
    geom_line(data=mean_per_hour, aes(x=time_range, y=Weekly), linewidth = 1.5)

For the weekdays, we can clearly see that the average loading evolves more or less in the same way. The stations are fuller in the morning, then people probably goes to work by bike between 8am and 10am. Many stations must have a really low loading rate at 8pm. In the weekend, the average loading is the lowest around 6pm. The velib are, in average, more used.


In [ ]:
noms_col <- names(load)

options(repr.plot.width = 13, repr.plot.height = 10)
hours = c(6, 12,15, 23)

dfi = coord
p1 = list()
for (i in 1:length(hours)){
    dfi$load = load[,hours[i]+1]
    p1[[i]] = ggplot(dfi, aes(x=longitude, y=latitude, color=load)) + 
        geom_point() +
        labs(title = paste("Stations loading - ",noms_col[hours[i]+1],"h"))
}

do.call(grid.arrange,c(p1, ncol=2))


We can see that early in the morning et late in the evening, the stations the busiest stations are all around Paris center. However, during the day, they are all along the Seine. We can imagine that people are taking velib to go to work and go back home. Moreover, when we took a look at the map of the cycle paths of Paris, we can see that most of them are along the Seine. We can suppose that people prefer to ride a bike on a cycle path and then leave the bike when the path stops to finish their routes by walk or public transport.


In [ ]:
options(repr.plot.width = 13, repr.plot.height = 10)
hours = c(126, 132, 143)

dfi = coord
p1 = list()
for (i in 1:length(hours)){
    dfi$load = load[,hours[i]+1]
    p1[[i]] = ggplot(dfi, aes(x=longitude, y=latitude, color=load)) + 
        geom_point() +
        labs(title = paste("Stations loading - ",noms_col[hours[i]+1],"h"))
}

do.call(grid.arrange,c(p1, ncol=2))

For the weekend, here the saturday, we can see that the velibs are globally more used. However, we can imagine that people are not going far from their place because the busiest stations stay the same more or less throughout the day. The stations are even busier at 6am, we can deduce that people are not going outside that early in the weekend.


In [ ]:
Hill <- load[coord$bonus == 1, ]
NoHill <- load[coord$bonus == 0, ]
options(repr.plot.width = 15, repr.plot.height = 6)


day <- c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")
HourHill <- matrix(colMeans(Hill), nrow = 24) # average loading of a station on the hill per hour for one day
HourNoHill <- matrix(colMeans(NoHill), nrow = 24) # average loading of a station not on the hill per hour for one day
options(repr.plot.width = 15, repr.plot.height = 6)
par(mfrow = c(1, 2))
matplot(HourHill, type = "l", xlim = c(0,26), ylim = c(0, 0.45), col = 1:7, pch = 19, 
        xlab = "Hour, stations on a hill", ylab = "Loading")
legend(x = 10, y = 0.47, day,lty = 1:7, col = 1:7)
matplot(HourNoHill, type = "l", ylim = c(0, 0.45), col = 1:7, pch = 19, 
        xlab = "Hour, stations not on a hill", ylab = "Loading")
legend(x = 9, y = 0.3, day, lty = 1:7, col = 1:7)


When we compare the stations that are located on a hill and those that are not on a hill, we can see a clear difference of the loading. During the weekdays, the loading of the stations of a hill is really low (between 0 et 20 %). People must use the bikes to go down to work and take the public transports to go back up, as we said earlier. During the weekend, those stations are always almost empty. However, for the stations that are not on a hill, the average loading per hour is higher (between 30 et 50%) and stays always in same range. 

## PCA



In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)
pca = PCA(load, scale.unit = TRUE,ncp =20, graph=FALSE)

# Visualisation of explained variance and variables
grid.arrange(fviz_eig(pca),fviz_pca_var(pca,axes=c(1,2)),ncol=2)

head(pca$eig)


We chose to keep four components so it explains enough the variances (72.9%) but not more to limit the number of components since it does not make us win a lot of explained variances.


In [ ]:
pca<-PCA(load, scale.unit = T, ncp = 4, graph = F)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 20)

df = as.data.frame(pca$svd$V)

df$time_range <- 1:ncol(load)

df <- reshape2::melt(df, id.vars = "time_range")


p1 = ggplot(df[df$variable == paste0("V", 1), ], aes(x=time_range, y= value)) + geom_line() +
    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
    labs(x = "Time", y = paste("Comp", 1, "Value"))

p2 = ggplot(df[df$variable == paste0("V", 2), ], aes(x=time_range, y= value)) + geom_line() +
    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
    labs(x = "Time", y = paste("Comp", 2, "Value"))

p3 = ggplot(df[df$variable == paste0("V", 3), ], aes(x=time_range, y= value)) + geom_line() +
    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
    labs(x = "Time", y = paste("Comp", 3, "Value"))

p4 = ggplot(df[df$variable == paste0("V", 4), ], aes(x=time_range, y= value)) + geom_line() +
     geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
     labs(x = "Time", y = paste("Comp", 4, "Value"))

#p5 = ggplot(df[df$variable == paste0("V", 5), ], aes(x=time_range, y= value)) + geom_line() +
#    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
#    labs(x = "Time", y = paste("Comp", 5, "Value"))



p <- list(p1,p2,p3,p4)
do.call(grid.arrange,p)


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 6)

grid.arrange(
  
    fviz_pca_ind(pca, axes=c(1,2), geom=c("point")),
    fviz_pca_ind(pca, axes=c(1,3), geom=c("point")),
    fviz_pca_ind(pca, axes=c(2,3), geom=c("point")),
    ncol=3
)

## Clustering the initial data

### Kmeans

Choice of the best K:

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 6)


fviz_nbclust(load, FUNcluster=kmeans, method="wss") +
    ggtitle("Within sum of square (WSS) according to the number of clusters")

fviz_nbclust(load, FUNcluster=kmeans, method="silhouette") +
    ggtitle("Silhouette score according to the number of clusters")

According to the silhouette score, we would choose k = 2, but according to WSS, it is more difficult to determine.


In [ ]:
library(factoextra)
library(cluster)
options(repr.plot.width = 15, repr.plot.height = 12)

# With 2 clusters
reskmeans = kmeans(load, centers=2) 
sil = silhouette(reskmeans$cluster, dist(load))
p1 = fviz_silhouette(sil)

# With 3 clusters
reskmeans = kmeans(load, centers=3) 
sil = silhouette(reskmeans$cluster, dist(load))
p2 = fviz_silhouette(sil)

# With 4 clusters
reskmeans = kmeans(load, centers=4) 
sil = silhouette(reskmeans$cluster, dist(load))
p3 = fviz_silhouette(sil)

# With 5 clusters
reskmeans = kmeans(load, centers=5) 
sil = silhouette(reskmeans$cluster, dist(load))
p4 = fviz_silhouette(sil)

grid.arrange(p1,p2,p3, p4,ncol=2)

This confirms our choice of k = 2.

In [ ]:
reskmeans = kmeans(load, centers=2) 

fviz_cluster(reskmeans, data=load, ellipse.type="norm", labelsize=8, geom=c("point"))

In [ ]:
tbl = table(coord$bonus, reskmeans$cluster)

options(repr.plot.width = 9, repr.plot.height = 6)
mosaicplot(tbl, color=c(2:4))


With this graph, we can see that the stations located on a hill are all in the same cluster, the first one. The ones not on a hill are almost equally divided between the two clusters.


In [ ]:
options(repr.plot.width = 6, repr.plot.height = 20)

p1 <- ggplot(load, aes(x = coord$longitude, y = coord$latitude)) +
  geom_point(aes(color = factor(reskmeans$cluster))) + 
  labs(title = "Clusters of the stations of velib", x = "Longitude", y = "Latitude") +
  theme_minimal()

dfi$load = load[,6]
p2 = ggplot(dfi, aes(x=longitude, y=latitude, color=load)) + 
        geom_point() +
        labs(title = paste("Stations loading - Monday 6 h"))

p = list(p1,p2)
do.call(grid.arrange,p)

By ploting the clusters by their location, we can see that they correspond more or less to the loading of the stations early in the morning and late in the evening.

### CAH


In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

grid.arrange(
    fviz_nbclust(load, FUNcluster=hcut, method="wss") + ggtitle("WSS according to nb of clusters"),
    fviz_nbclust(load, FUNcluster=hcut, method="silhouette") + ggtitle("silhouette according to nb of clusters"),
    ncol=2
)

In [ ]:
d = dist(load, method="euclidean")
hclustcomplete = hclust(d, method="complete")
reshclust = cutree(hclustcomplete, 2)

# --- #

fviz_dend(hclustcomplete, k=2, show_labels=FALSE, rect=TRUE)

In [ ]:
tbl = table(coord$bonus, reshclust)
print(tbl)

options(repr.plot.width = 9, repr.plot.height = 6)
mosaicplot(tbl, color=c(2:4))

Once again, the stations located on a hill are all in the first cluster. However, the clusters are very unequal in size.


In [ ]:
p1 <- ggplot(load, aes(x = coord$longitude, y = coord$latitude)) +
  geom_point(aes(color = factor(reshclust))) +  
  labs(title = "Clusters des stations de métro", x = "Longitude", y = "Latitude") +
  theme_minimal()


dfi$load = load[,20]
p2 = ggplot(dfi, aes(x=longitude, y=latitude, color=load)) + 
        geom_point() +
        labs(title = paste("Stations loading - Monday 20 h"))

dfi$load = load[,6]
p3 = ggplot(dfi, aes(x=longitude, y=latitude, color=load)) + 
        geom_point() +
        labs(title = paste("Stations loading - Monday 6 h"))

p <- list(p1,p2,p3)

do.call(grid.arrange,p)


The clusters of the kmeans method and CAH method are similar. 

###GMM

In [ ]:
library(mclust)
#resBICall = mclustBIC(load, G=2:20)
#summary(resBICall)

# --- #

#resBICall = Mclust(load, G=2:20)
#summary(resBICall)

#fviz_mclust(resBICall, what="BIC")


The loading of thecode with so much data takes too long, so we'll only use this method with PCA.

## Clustering after PCA

### Kmeans

Choice of k:

In [ ]:
pca_data <- pca$ind$coord
options(repr.plot.width = 10, repr.plot.height = 6)


fviz_nbclust(pca_data, FUNcluster=kmeans, method="wss") +
    ggtitle("Within sum of square (WSS) according to the number of clusters")

fviz_nbclust(pca_data, FUNcluster=kmeans, method="silhouette") +
    ggtitle("Silhouette score according to the number of clusters")


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 12)

# With 2 clusters
reskmeans = kmeans(pca_data, centers=2) 
sil = silhouette(reskmeans$cluster, dist(pca_data))
p1 = fviz_silhouette(sil)

# With 3 clusters
reskmeans = kmeans(pca_data, centers=3) 
sil = silhouette(reskmeans$cluster, dist(pca_data))
p2 = fviz_silhouette(sil)

# With 4 clusters
reskmeans = kmeans(pca_data, centers=4) 
sil = silhouette(reskmeans$cluster, dist(pca_data))
p3 = fviz_silhouette(sil)

# With 5 clusters
reskmeans = kmeans(pca_data, centers=5) 
sil = silhouette(reskmeans$cluster, dist(pca_data))
p4 = fviz_silhouette(sil)

grid.arrange(p1,p2,p3, p4,ncol=2)

Here, we can choose k = 2.

In [ ]:
reskmeans2 = kmeans(pca_data, centers=2) 

fviz_cluster(reskmeans2, data=pca_data, ellipse.type="norm", labelsize=8, geom=c("point"))

In [ ]:
tbl = table(coord$bonus, reskmeans2$cluster)
print(tbl)

options(repr.plot.width = 9, repr.plot.height = 6)
mosaicplot(tbl, color=c(2:4))

Once more, the stations located on a hill are all in the same cluster, the second one here.


In [ ]:
p1 <- ggplot(pca_data, aes(x = coord$longitude, y = coord$latitude)) +
  geom_point(aes(color = factor(reskmeans2$cluster))) +
  labs(title = "Clusters des stations de métro", x = "Longitude", y = "Latitude") +
  theme_minimal()

dfi$load = load[,12]
p2 = ggplot(dfi, aes(x=longitude, y=latitude, color=load)) + 
        geom_point() +
        labs(title = paste("Stations loading - Monday 12 h"))

p = list(p1,p2)
do.call(grid.arrange,p)

Those clusters are more close to the loading of the stations at 12h. The first and third clusters are difficult to interpret.

### CAH

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

grid.arrange(
    fviz_nbclust(pca_data, FUNcluster=hcut, method="wss") + ggtitle("WSS according to nb of clusters"),
    fviz_nbclust(pca_data, FUNcluster=hcut, method="silhouette") + ggtitle("silhouette according to nb of clusters"),
    ncol=2
)

Once more, we choose 3 clusters.

In [ ]:
reshclust = cutree(hclustcomplete, 3)

# --- #

fviz_dend(hclustcomplete, k=3, show_labels=FALSE, rect=TRUE)

In [ ]:
tbl = table(coord$bonus, reshclust)
print(tbl)

options(repr.plot.width = 9, repr.plot.height = 6)
mosaicplot(tbl, color=c(2:4))


In [ ]:
p1 <- ggplot(pca_data, aes(x = coord$longitude, y = coord$latitude)) +
  geom_point(aes(color = factor(reshclust))) +  # Utilisation de 'Cluster' pour la couleur
  labs(title = "Clusters des stations", x = "Longitude", y = "Latitude") +
  theme_minimal()

print(p1)

The clusters of the CAH and kmeans methods are similar.


### GMM 


In [ ]:
library(mclust)
resBICall = mclustBIC(pca_data, G=2:7)
summary(resBICall)

# --- #

resBICall = Mclust(pca_data, G=2:7)
summary(resBICall)

fviz_mclust(resBICall, what="BIC")

In [ ]:

gmm_mod <- Mclust(data = pca_data, G=7, nstart=10)

options(repr.plot.width = 10, repr.plot.height = 10)
p2 <- ggplot(pca_data, aes(x = coord$longitude, y = coord$latitude)) +
  geom_point(aes(color = factor(gmm_mod$classification))) +  # Utilisation de 'Cluster' pour la couleur
  labs(title = "Clusters des stations", x = "Longitude", y = "Latitude") +
  theme_minimal()

print(p2)

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 7)
print(fviz_cluster(object=gmm_mod,data=compo_acp,show.clust.cent=TRUE,ellipse=TRUE))

In [ ]:
nc_gmm = 7
time_range = 1:ncol(load)

options(repr.plot.width = 28, repr.plot.height =12)


for (i in 1:nc_gmm){
    loadgmm <- load[gmm_mod$classification==i,]
    plot(time_range,colMeans(loadgmm), type = "l")}